<a href="https://colab.research.google.com/github/anskong/ai_ref_from_lessons/blob/main/LGCNS_LangChain_015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검색결과를 이용한 답변 생성(PDF 업로드 → 텍스트 분할 → 벡터화 → RAG 질문응답)


In [ ]:
# 1) 필요한 패키지 설치
!pip install --quiet --upgrade "langchain-core>=0.3.0" "langchain-community>=0.0.30" "langchain-openai>=0.0.8" "langchain-chroma>=0.0.5" "langchain-text-splitters>=0.0.1" openai chromadb pypdf

In [ ]:
# 2) Colab 파일 업로드
from google.colab import files
import os
from google.colab import userdata

# 3) OPENAI API 키 불러오기
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("📤 PDF 파일을 업로드하세요...")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 4) LangChain 구성요소 임포트
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage
from operator import itemgetter

# 5) 문서 로딩 및 분할
loader = PyPDFLoader(filename)
pages = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
splits = splitter.split_documents(pages)

# 6) Chroma 벡터 스토어 생성
persist_directory = "chroma_db"
embedding = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory=persist_directory)

# 7) 리트리버 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 8) 프롬프트 템플릿 정의
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage("당신은 유능한 조수입니다."),
    MessagesPlaceholder("chat_history"),
    HumanMessagePromptTemplate.from_template(
        "주어진 컨텍스트에 따라 다음 질문에 답하십시오.\n컨텍스트：{context}\n질문：{question}"
    )
])

# 9) LLM 및 출력 파서 정의
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
parser = StrOutputParser()

# 10) 문서 포맷터
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 11) LCEL 기반 RAG 체인 구성
rag_chain = (
    {
        "context": itemgetter("question") | retriever | format_docs,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt_template
    | llm
    | parser
)

# 12) 체인 실행
history = []
question = "생성형 AI의 최신동향은?"
answer = rag_chain.invoke({"question": question, "chat_history": history})

# 13) 결과 출력
print(f"\n🧠 질문: {question}")
print("📝 답변:\n", answer)

📤 PDF 파일을 업로드하세요...


Saving 2024_한국인터넷백서_펼침.pdf to 2024_한국인터넷백서_펼침.pdf

🧠 질문: 생성형 AI의 최신동향은?
📝 답변:
 생성형 AI 기술은 계속해서 발전하고 있습니다. 최근에는 영상 생성 인공지능 기술이 주목을 받고 있습니다. 2024년에는 구글의 '루미에르(Lumiere)'와 오픈AI의 '소라(Sora)' 같은 고화질 영상 생성 AI가 소개되었으며, 이러한 기술들은 더욱 발전하여 초고해상도의 영상을 생성하고 더욱 빠르게 작업할 수 있는 기능을 제공하고 있습니다. 또한 중국의 기업들도 생성형 AI 기술에 투자하고 있으며, 이를 통해 AI를 활용한 콘텐츠 제작이 보다 쉽고 다양해지고 있습니다. 이러한 발전은 콘텐츠 시장과 산업에 새로운 가능성을 제시하고 있습니다.
